# **TAREA LanceDB**
- Considera usar ANN para cada búsqueda o filtro

**Task 1: Consulta avanzada con proyección y filtro**

Instrucciones:
1. Genera un vector aleatorio con la misma dirección que los embeddings que están en la tabla mis_vectores
2. Realiza una búsqueda en la tabla para encontrar los 5 elementos más cercanos
3. Proyecta los resultados para mostrar solo las columnas item y _distance
4. Excluye de los resultado los elementos cuyo nombre sea 'item 500'

Pregunta: ¿Cuáles son los cinco elementos más cercanos que cumplen con los criterios y cuál es la distancia de cada uno?

In [11]:
# Cargar librerías necesarias
import lancedb
import numpy as np
import pandas as pd
# Configurar LanceDB
uri = "data/ANN"  # Directorio donde se guardará la base de datos
db = lancedb.connect(uri)

# Crear 10,000 vectores de muestra aleatorios
np.random.seed(42)  # Para reproducibilidad
data = [
    {"vector": row, "item": f"item {i}"}
    for i, row in enumerate(np.random.random((10_000, 1536)).astype("float32")) #Le puse solo 10,000 pq mi compu no daba pa más
]

# Crear tabla en la base de datos
tbl = db.create_table("mis_vectores", data=data, mode = "overwrite")
print(f"Tabla 'mis_vectores' creada con {len(data)} vectores.")

Tabla 'mis_vectores' creada con 10000 vectores.


In [12]:
import lancedb
import numpy as np
import pandas as pd

# Conectar a la base de datos existente
uri = "data/ANN"  # Directorio donde se encuentra la base de datos
db = lancedb.connect(uri)
table = db.open_table("mis_vectores")  # Abrimos la tabla "mis_vectores"

# Paso 1: Generar un vector aleatorio de la misma dimensión (1536)
np.random.seed(42)  # Opcional para reproducibilidad
vector_aleatorio = np.random.random(1536).astype("float32")

# Paso 2: Realizar la búsqueda ANN para los 5 elementos más cercanos
resultados = table.search(vector_aleatorio).limit(5).to_pandas()

# Mostrar los resultados
print("Los 5 elementos más cercanos al vector de consulta son:")
print(resultados)

# Paso 3: Proyectar solo las columnas item y _distance
resultados_proyectados = resultados[["item", "_distance"]]

# Mostrar los resultados proyectados
print("\nResultados proyectados (item y _distance):")
print(resultados_proyectados)

# Paso 4: Excluir elementos cuyo nombre sea "item 500"
resultados_filtrados = resultados_proyectados[resultados_proyectados["item"] != "item 500"]

# Seleccionar los 5 primeros resultados tras el filtro
resultados_finales = resultados_filtrados.head(5)

# Mostrar los resultados finales
print("\nPregunta: ¿Cuáles son los cinco elementos más cercanos que cumplen con los criterios y cuál es la distancia de cada uno?")
print("\nResultados finales tras excluir 'item 500':")
print(resultados_finales)

[2024-12-05T17:57:10Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2024-12-05T17:57:12Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2024-12-05T17:57:12Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2024-12-05T17:57:15Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.


Los 5 elementos más cercanos al vector de consulta son:
                                              vector       item   _distance
0  [0.37454012, 0.9507143, 0.7319939, 0.5986585, ...     item 0    0.000000
1  [0.20024236, 0.60035306, 0.2857333, 0.39349523...  item 1523  230.041855
2  [0.47765255, 0.26411608, 0.45712984, 0.9785217...  item 7241  234.380585
3  [0.7418995, 0.510742, 0.121364646, 0.2387483, ...  item 3127  236.566757
4  [0.4597147, 0.96161556, 0.42014694, 0.99712867...  item 8664  236.600876

Resultados proyectados (item y _distance):
        item   _distance
0     item 0    0.000000
1  item 1523  230.041855
2  item 7241  234.380585
3  item 3127  236.566757
4  item 8664  236.600876

Pregunta: ¿Cuáles son los cinco elementos más cercanos que cumplen con los criterios y cuál es la distancia de cada uno?

Resultados finales tras excluir 'item 500':
        item   _distance
0     item 0    0.000000
1  item 1523  230.041855
2  item 7241  234.380585
3  item 3127  236.566757
4 

**Task 2: Creación de tablas**

Instrucciones:
1. Define un nuevo esquema para una tabla vacía con las siguientes columnas:
*   vector (vector de 4 dimensiones)
*   nombre
*   categoria
2. Crea una tabla vacía llamada nueva_tabla usando el esquema
3. Inserta 5 registros en la tabla
4. Muestra el contenido de la tabla





In [3]:
import lancedb
import pandas as pd
import pyarrow as pa
import polars as pl
import numpy as np
import random
from lancedb.pydantic import Vector, LanceModel
from pydantic import BaseModel

# Definir el esquema usando LanceModel
class CustomSchema(LanceModel):
    vector: Vector(4)  # Vector de 4 dimensiones
    nombre: str        # Nombre como cadena
    categoria: str     # Categoría como cadena

# Conectar o crear la base de datos
uri = "data/ANN"  # Directorio donde se guardará la base de datos
db = lancedb.connect(uri)

# Crear la tabla con el esquema definido
tabla = db.create_table("nueva_tabla", schema=CustomSchema, mode="overwrite")

# Validar el esquema de la tabla
print(f"Tabla '{tabla.name}' creada con éxito con el esquema:")
print(tabla.schema)  # Usar sin paréntesis para mostrar el esquema

# Lista de datos para insertar
items = [
    {"vector": [0.1, 0.2, 0.3, 0.4], "nombre": "Laptop", "categoria": "Electrónica"},
    {"vector": [0.3, 0.1, 0.4, 0.5], "nombre": "Audífonos", "categoria": "Electrónica"},
    {"vector": [0.5, 0.2, 0.1, 0.3], "nombre": "Smartphone", "categoria": "Electrónica"},
    {"vector": [0.6, 0.7, 0.8, 0.9], "nombre": "Escritorio", "categoria": "Muebles"},
    {"vector": [0.9, 0.8, 0.7, 0.6], "nombre": "Silla", "categoria": "Muebles"}
]

# Insertar datos en la tabla
tabla.add(items)

# Verificar los datos insertados
df = tabla.to_pandas()
print(df)

[2024-12-05T17:48:50Z WARN  lance::dataset] No existing dataset at /usr/src/app/notebooks/data/ANN/nueva_tabla.lance, it will be created


Tabla 'nueva_tabla' creada con éxito con el esquema:
vector: fixed_size_list<item: float>[4] not null
  child 0, item: float
nombre: string not null
categoria: string not null
                 vector      nombre    categoria
0  [0.1, 0.2, 0.3, 0.4]      Laptop  Electrónica
1  [0.3, 0.1, 0.4, 0.5]   Audífonos  Electrónica
2  [0.5, 0.2, 0.1, 0.3]  Smartphone  Electrónica
3  [0.6, 0.7, 0.8, 0.9]  Escritorio      Muebles
4  [0.9, 0.8, 0.7, 0.6]       Silla      Muebles


[2024-12-05T17:48:50Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2024-12-05T17:48:50Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.


**Task 3: Actualización de vectores y filtrado**

Instrucciones:
1. Crea una tabla utilizando un DataFrame de Pandas con las siguientes columnas:
- id (Entero).
- vector (Lista de tres números flotantes)
2. Actualiza el vector de la fila donde id=3 a [10.0, 11.0, 10.0]
3. Filtra la tabla para mostrar solo las filas donde al menos un valor del vector sea mayor a 9.0


In [4]:
import pandas as pd
import lancedb

# Conectar o crear la base de datos
uri = "data/ANN"  # Directorio donde se guardará la base de datos
db = lancedb.connect(uri)

# Crear un DataFrame con las columnas id y vector
data = pd.DataFrame({
    "id": [1, 2, 3, 4, 5],  # id como enteros
    "vector": [
        [1, 2, 3], 
        [4, 5, 6], 
        [7, 8, 9],
        [1.0, 11, 1.2],
        [1.3, 14, 15]
    ]
})

# Crear la tabla en LanceDB usando el DataFrame
table_pd = db.create_table("pd_table", data, exist_ok=True, mode="overwrite")

# Mostrar el esquema de la tabla para verificar que se creó correctamente
print(f"Tabla '{table_pd.name}' creada con éxito con el esquema:")
print(table_pd.schema)

# Actualizar el vector donde id = 3
table_pd.update(where="id == 3", values={"vector": [10.0, 11.0, 10.0]})

# Filtrar la tabla para mostrar solo las filas donde al menos un valor del vector sea mayor a 9.0
filtered_df = table_pd.to_pandas()

# Aplicar el filtro
filtered_df = filtered_df[filtered_df['vector'].apply(lambda x: any(i > 9.0 for i in x))]

# Mostrar los resultados filtrados
print(filtered_df)

[2024-12-05T17:49:01Z WARN  lance::dataset] No existing dataset at /usr/src/app/notebooks/data/ANN/pd_table.lance, it will be created


Tabla 'pd_table' creada con éxito con el esquema:
id: int64
vector: fixed_size_list<item: float>[3]
  child 0, item: float


[2024-12-05T17:49:01Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2024-12-05T17:49:01Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2024-12-05T17:49:01Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2024-12-05T17:49:01Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.


   id              vector
2   4    [1.0, 11.0, 1.2]
3   5   [1.3, 14.0, 15.0]
4   3  [10.0, 11.0, 10.0]


[2024-12-05T17:49:01Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2024-12-05T17:49:01Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.


**Task 4: Embeddings multimodales y búsqueda combinada**

Instrucciones:
1. Crea una tabla con datos de texto e imágenes combinados. Incluye las siguientes columnas:
- texto (Texto).
- imagen (Nombre del archivo de imagen).
- embedding_texto (Vector del texto generado con SentenceTransformer).
- embedding_imagen (Vector de la imagen generado con ResNet18).
2. Realiza una consulta para encontrar los elementos con un texto similar a "La tecnología avanza rápido" y una imagen visualmente similar a un color predominantemente azul.
3. Muestra los resultados combinados ordenados por la menor distancia promedio entre ambos embeddings.


In [5]:
# Parte 1
import lancedb
import torch
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from PIL import Image
from torchvision.models import resnet18, ResNet18_Weights
from torchvision import transforms
import os
import pyarrow as pa
from sklearn.metrics.pairwise import cosine_similarity

# Configurar LanceDB
db_path = "data/vectorial_db"  # Ruta donde se almacenará la base de datos

# Crear el directorio si no existe
os.makedirs(db_path, exist_ok=True)

# Conectar a LanceDB
db = lancedb.connect(db_path)
print(f"Base de datos configurada en: {db_path}")

# Cargar modelo para embeddings de texto
text_model = SentenceTransformer('all-MiniLM-L6-v2')

# Datos de texto de ejemplo
text_data = [
    "El cielo es azul y hermoso.",
    "Los gatos son mascotas populares.",
    "La inteligencia artificial está transformando el mundo.",
    "Las flores son coloridas y fragantes.",
    "Viajar es una experiencia enriquecedora.",
    "La tecnología avanza rápido"
]

# Generar embeddings de texto
text_embeddings = text_model.encode(text_data)
text_embeddings = [embedding.astype(np.float32).tolist() for embedding in text_embeddings]  # Convertir a float32

# Crear DataFrame con los datos y embeddings de texto
df_text = pd.DataFrame({
    "texto": text_data,
    "embedding_texto": text_embeddings
})

# Configuración del modelo de imagen ResNet18
image_model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
image_model.eval()

# Configuración de la transformación de imágenes
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Ruta de la carpeta de imágenes
image_dir = "data/images"

# Procesar las imágenes y generar embeddings
image_embeddings = []
image_data = []

for image_file in os.listdir(image_dir):
    if image_file.endswith(".jpg") or image_file.endswith(".png"):
        img_path = os.path.join(image_dir, image_file)
        img = Image.open(img_path).convert("RGB")
        img_tensor = image_transform(img).unsqueeze(0)

        # Generar embedding de la imagen
        with torch.no_grad():
            embedding = image_model(img_tensor).squeeze().numpy().astype('float32')

        image_data.append(image_file)  # Nombre de la imagen
        image_embeddings.append(embedding.tolist())  # Embedding como lista

# Crear DataFrame con los datos y embeddings de imagen
df_images = pd.DataFrame({
    "imagen": image_data,
    "embedding_imagen": image_embeddings
})

# Crear DataFrame combinado de texto e imagen
df_combined = pd.merge(df_text, df_images, left_index=True, right_index=True, how='outer')

# Crear esquema Arrow con las columnas 'texto', 'imagen', 'embedding_texto', 'embedding_imagen'
schema = pa.schema([
    ("texto", pa.string()),  # Columna de texto
    ("imagen", pa.string()),  # Columna de imagen
    ("embedding_texto", pa.list_(pa.float32(), list_size=len(text_embeddings[0]))),  # Embedding de texto
    ("embedding_imagen", pa.list_(pa.float32(), list_size=len(image_embeddings[0])))  # Embedding de imagen
])

# Convertir el DataFrame combinado en una tabla Arrow
table_combined = pa.Table.from_pandas(df_combined, schema=schema)

# Guardar la tabla combinada en LanceDB
tabla_combined = db.create_table("multimodal_embeddings", data=table_combined, mode="overwrite")
print("Tabla 'multimodal_embeddings' creada con éxito.")

# Parte 2: Consultas y cálculo de distancia promedio

# 1. Consulta de texto: Similaridad con "La tecnología avanza rápido"
query_text = "La tecnología avanza rápido"  # Texto de consulta
query_text_embedding = text_model.encode([query_text])[0].astype(np.float32)  # Generar embedding

# 2. Consulta de imagen: Similaridad con color predominantemente azul
query_image = Image.new("RGB", (224, 224), (50, 50, 255))  # Color RGB con predominancia azul
query_image_tensor = image_transform(query_image).unsqueeze(0)

# Generar embedding para la imagen de consulta
with torch.no_grad():
    query_image_embedding = image_model(query_image_tensor).squeeze().numpy().astype('float32')

# Recuperar datos de la tabla para procesarlos
results = tabla_combined.to_pandas()

# Filtrar filas donde ambos embeddings no sean None
results = results.dropna(subset=["embedding_texto", "embedding_imagen"]).reset_index(drop=True)

# Calcular distancia promedio entre embeddings
def calculate_combined_distance(row):
    text_distance = np.linalg.norm(np.array(row["embedding_texto"]) - query_text_embedding)
    image_distance = np.linalg.norm(np.array(row["embedding_imagen"]) - query_image_embedding)
    return (text_distance + image_distance) / 2

# Aplicar cálculo de distancia
results["distance"] = results.apply(calculate_combined_distance, axis=1)

# Ordenar por la menor distancia promedio
results = results.sort_values("distance").reset_index(drop=True)

# Mostrar los resultados combinados
print("Resultados ordenados por menor distancia promedio:")
print(results[["texto", "imagen", "distance"]])

Base de datos configurada en: data/vectorial_db


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:05<00:00, 8.22MB/s]
[2024-12-05T17:51:09Z WARN  lance::dataset] No existing dataset at /usr/src/app/notebooks/data/vectorial_db/multimodal_embeddings.lance, it will be created


Tabla 'multimodal_embeddings' creada con éxito.
Resultados ordenados por menor distancia promedio:
                                               texto  \
0           Viajar es una experiencia enriquecedora.   
1                  Los gatos son mascotas populares.   
2                        La tecnología avanza rápido   
3              Las flores son coloridas y fragantes.   
4  La inteligencia artificial está transformando ...   
5                        El cielo es azul y hermoso.   

                             imagen   distance  
0  lancedb_embedded_explanation.png  34.785149  
1                        ivf_pq.png  34.935299  
2                      imagen_3.jpg  38.116432  
3                      imagen_4.jpg  38.346001  
4                      imagen_6.jpg  39.997059  
5                      imagen_2.jpg  45.970905  


[2024-12-05T17:51:09Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2024-12-05T17:51:09Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
